In [2]:
# imports
import pandas as pd
import numpy as np
import datetime
import dateutil.relativedelta

import yfinance as yf
import yahoo_finance as yhf
from bcb import sgs
import pypfopt
from pypfopt import risk_models
from pypfopt import expected_returns
import cvxpy

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import mplfinance as mpf
import matplotlib.dates as mpl_dates

import sqlite3
import os
import logging

In [3]:
# Conexão com base local de dados
os.chdir(os.getcwd())
conn = sqlite3.connect(r"C:\databases\assets_portfolio_pessoal.db")
cur = conn.cursor()

In [4]:
# datas
# datas

def meses_para_tras(meses):
    hoje = datetime.datetime.today() - datetime.timedelta(days=1)
    data = hoje - dateutil.relativedelta.relativedelta(months=meses)
    return data

hoje = meses_para_tras(0)

datas_train = (meses_para_tras(12*5), meses_para_tras(12))
datas_teste = (meses_para_tras(12), hoje)

In [5]:
# Todas ações B3
acoes_listadas_b3_csv = pd.read_csv('acoes-listadas-b3.csv')

tickers_b3 = [*acoes_listadas_b3_csv['Ticker'].apply(lambda x: x+'.SA')]
data_b3 = yf.download(tickers_b3, end=hoje)

data_b3['Adj Close'].to_sql("b3_adj_close", conn, if_exists="replace", index=True)
data_b3['Volume'].to_sql("b3_volume", conn, if_exists="replace", index=True)
data_b3['Adj Close'].pct_change().dropna().to_sql("b3_retornos", conn, if_exists="replace", index=True)

[                       0%                       ]  2 of 402 completed

C:\Users\loren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:689: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
C:\Users\loren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:689: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
C:\Users\loren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:689: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a fu

[*                      2%                       ]  8 of 402 completed

C:\Users\loren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:689: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
C:\Users\loren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:689: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
C:\Users\loren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:689: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a fu

[*                      3%                       ]  12 of 402 completed

...

[**********************99%***********************]  398 of 402 completed

C:\Users\loren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:689: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
C:\Users\loren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:689: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
C:\Users\loren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:689: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a fu

[*********************100%***********************]  402 of 402 completed

4 Failed downloads:
- BIOM11.SA: No timezone found, symbol may be delisted
- AMAR11.SA: No timezone found, symbol may be delisted
- AZEV11.SA: No timezone found, symbol may be delisted
- PINE11.SA: No timezone found, symbol may be delisted


C:\Users\loren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:689: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
C:\Users\loren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:689: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
C:\Users\loren\AppData\Local\Temp\ipykernel_11216\2118338226.py:9: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or speci

0

In [6]:
# Todas ações IBOVESPA
ibov = pd.read_csv('IBOVQuad_5-2024.csv', sep=';', header=1, index_col=False)

tickers_ibov = [*ibov['C�digo'].apply(lambda x: x+'.SA')]
data_ibov = yf.download(tickers_b3, end=hoje)

data_ibov['Adj Close'].to_sql("ibov_adj_close", conn, if_exists="replace", index=True)
data_ibov['Volume'].to_sql("ibov_volume", conn, if_exists="replace", index=True)
data_ibov['Adj Close'].pct_change().dropna().to_sql("ibov_retornos", conn, if_exists="replace", index=True)

[                       1%                       ]  3 of 402 completed

C:\Users\loren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:689: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
C:\Users\loren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:689: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
C:\Users\loren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:689: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a fu

[                       1%                       ]  6 of 402 completed

C:\Users\loren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:689: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
C:\Users\loren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:689: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
C:\Users\loren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:689: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a fu

[*                      2%                       ]  10 of 402 completed

...

[*********************100%***********************]  402 of 402 completed

C:\Users\loren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:689: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
C:\Users\loren\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\yfinance\utils.py:689: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')




4 Failed downloads:
- BIOM11.SA: No timezone found, symbol may be delisted
- AMAR11.SA: No timezone found, symbol may be delisted
- AZEV11.SA: No timezone found, symbol may be delisted
- PINE11.SA: No timezone found, symbol may be delisted


C:\Users\loren\AppData\Local\Temp\ipykernel_11216\2688852769.py:9: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  data_ibov['Adj Close'].pct_change().dropna().to_sql("ibov_retornos", conn, if_exists="replace", index=True)


0

In [7]:
conn.close()